In [1]:
from transformers import pipeline
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

classifier = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2", device=device
)

/home/gautam/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/gautam/.local/lib/python3.8/site-packages/transformers/audio_utils.py:198: UserWarning: At least one mel filter has all zero values. The value for `num_mel_filters` (128) may be set too high. Or, the value for `num_frequency_bins` (256) may be set too low.
  warnings.warn(


In [2]:
classifier.model.config.id2label
classifier.model.config.id2label[27]

'marvin'

In [3]:
from transformers.pipelines.audio_utils import ffmpeg_microphone_live


def launch_fn(
    wake_word="marvin",
    prob_threshold=0.5,
    
    chunk_length_s=2.0,
    stream_chunk_s=0.25,
    debug=False,
):
    if wake_word not in classifier.model.config.label2id.keys():
        raise ValueError(
            f"Wake word {wake_word} not in set of valid class labels, pick a wake word in the set {classifier.model.config.label2id.keys()}."
        )

    sampling_rate = classifier.feature_extractor.sampling_rate

    mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )

    print("Listening for wake word...")
    for prediction in classifier(mic):
        prediction = prediction[0]
        if debug:
            print(prediction)
        if prediction["label"] == wake_word:
            if prediction["score"] > prob_threshold:
                return True

In [4]:
launch_fn(debug=True)

Listening for wake word...
{'score': 0.05387924984097481, 'label': 'two'}
{'score': 0.05484699457883835, 'label': 'up'}
{'score': 0.06849587708711624, 'label': 'up'}
{'score': 0.05104025825858116, 'label': 'off'}
{'score': 0.053439151495695114, 'label': 'off'}
{'score': 0.052375514060258865, 'label': 'off'}
{'score': 0.052375514060258865, 'label': 'off'}
{'score': 0.052375514060258865, 'label': 'off'}
{'score': 0.055426232516765594, 'label': 'up'}
{'score': 0.07175736129283905, 'label': 'four'}
{'score': 0.06757525354623795, 'label': 'stop'}
{'score': 0.06757525354623795, 'label': 'stop'}
{'score': 0.06757525354623795, 'label': 'stop'}
{'score': 0.06757525354623795, 'label': 'stop'}
{'score': 0.05650804936885834, 'label': 'tree'}
{'score': 0.05748778209090233, 'label': 'tree'}
{'score': 0.05748778209090233, 'label': 'tree'}
{'score': 0.05748778209090233, 'label': 'tree'}
{'score': 0.05748778209090233, 'label': 'tree'}
{'score': 0.06109370291233063, 'label': 'tree'}
{'score': 0.06491079

True

In [5]:
transcriber = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base.en", device=device
)

In [6]:
import sys


def transcribe(chunk_length_s=5.0, stream_chunk_s=1.0):
    sampling_rate = transcriber.feature_extractor.sampling_rate

    mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )

    print("Start speaking...")
    for item in transcriber(mic, generate_kwargs={"max_new_tokens": 128}):
        sys.stdout.write("\033[K")
        print(item["text"], end="\r")
        if not item["partial"][0]:
            break

    return item["text"]

In [7]:
transcribe()

Start speaking...


' Hello Marvin'

In [8]:
from huggingface_hub import HfFolder
import requests


def query(text, model_id="tiiuae/falcon-7b-instruct"):
    api_url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {HfFolder().get_token()}"}
    payload = {"inputs": text}

    print(f"Querying...: {text}")
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()[0]["generated_text"][len(text) + 1 :]

In [9]:
query("What does Hugging Face do?")


Querying...: What does Hugging Face do?


KeyError: 0

In [26]:
!pip install sentencepiece


     |████████████████████████████████| 1.3 MB 4.7 MB/s eta 0:00:01


In [10]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)

In [11]:
from datasets import load_dataset

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Generating validation split: 100%|██████████| 7931/7931 [00:00<00:00, 55013.52 examples/s]


In [12]:
def synthesise(text):
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(
        inputs["input_ids"].to(device), speaker_embeddings.to(device), vocoder=vocoder
    )
    return speech.cpu()

In [1]:
from IPython.display import Audio

audio = synthesise(
    "hurrah, we won the game"
)

Audio(audio, rate=16000)

NameError: name 'synthesise' is not defined

In [23]:
from transformers import HfAgent

agent = HfAgent(
    url_endpoint="https://api-inference.huggingface.co/models/bigcode/starcoder"
)

In [24]:
agent.run("Generate an image of a cat")

ValueError: Error 400: {'error': 'Authorization header is correct, but the token seems invalid'}

In [25]:
launch_fn()
transcription = transcribe()
agent.run(transcription)

Listening for wake word...
Start speaking...


ValueError: Error 400: {'error': 'Authorization header is correct, but the token seems invalid'}